# Develop notebook
Develop notebook for issue with staled training in scratch networks for Udacity [Deep Learning nanodegree](https://udacity.com/course/deep-learning-nanodegree--nd101), [dog-breed classifier](https://github.com/udacity/deep-learning-v2-pytorch/tree/master/project-dog-classification) project.

## Setup

#### Imports

In [1]:
import os
import re
import numpy as np
from glob import glob
from PIL import ImageFile
# Set PIL to be tolerant of image files that are truncated.
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.optim as optim
import torch.nn.init as init
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets
import torchvision.transforms as transforms
import torchvision.models as models

### Data loading

#### Set up variables

In [2]:
batch_size = 32        # Number of samples per batch in DataLoader
num_workers = 4        # Number of DataLoder processes
n_classes = 133        # Number of outcome classes. Used for network setup
networks = {}          # Holds multiple networks to be trained
checkpoints = {}       # Checkpoint name and locations
TEST_IN_TRAIN = False   # If testing should be done in the training loop. Used for debuging/testing
image_size = 224       # Network used image size

## Train parameters
n_epochs = 200               #
loss_fn = "cross_entropy"    # Select loss function. Selects correct loss input preprocessing function for training and testing loops
lr = 0.001                   # Learning rate

#### Load data

In [23]:
# Check if CUDA is available
use_cuda = torch.cuda.is_available()


# Training transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
# Same transform for test and validation sets
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Data location>
data_dir = "dogImages/"

# Train, validation and test data location
train_dir = os.path.join(data_dir,"train/")
valid_dir = os.path.join(data_dir,"valid/")
test_dir = os.path.join(data_dir,"test/")

# Load and transform data
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)# + datasets.ImageFolder(train_dir, transform=train_transforms) + datasets.ImageFolder(train_dir, transform=train_transforms) # Increase sample threefold 
valid_data = datasets.ImageFolder(valid_dir, transform=test_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

loaders_scratch = {
    "train": torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers),
    "valid": torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True, num_workers=num_workers),
    "test" : torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
}

### Helper functions - `weight_init_normal`, `select_criterion`
`weight_init_normal` (from [here](https://gist.github.com/jeasinema/ed9236ce743c8efaf30fa2ff732749f5)) sets network layer weigths to normally distributed, and is optional. `select_criterion` enables fast switching between different loss functions without having to modify network architectures.

In [4]:
def weight_init_normal(model):
    '''
    Initialize network weigths to normal random distribution.
    From: https://gist.github.com/jeasinema/ed9236ce743c8efaf30fa2ff732749f5
    
    Args:
        model: an initialize model
    
    Returns: None
    
    Usage:
        model = Model()
        model.apply(weight_init)
    '''
    if isinstance(model, nn.Conv2d):
        init.xavier_normal_(model.weight.data)
        if model.bias is not None:
            init.normal_(model.bias.data)
    elif isinstance(model, nn.Linear):
        init.xavier_normal_(model.weight.data)
        init.normal_(model.bias.data)
    elif isinstance(model, nn.BatchNorm2d):
        init.normal_(model.weight.data, mean=1, std=0.02)
        init.constant_(model.bias.data, 0)

In [5]:
def select_criterion(lossfn="cross_entropy"):
    '''
    Helper function that selects valid loss and preprocessing (of output according to loss) functions. Some functions need
    data that is preprocessed in various ways, e.g. NLLLoss expects LogSoftMax input, but CrossEntropyLoss includes LogSoftMax.
    This selector simplifies changing loss function without having to rewrite the network arch, when testing various networks
    and loss functions.
    
    Args:
        fn: <fn> string with criterion name as in torch.nn.functional.<fn>
    
    Returns:
        preprocess: preprocess function for CNN output before passed to criterion. Defaults is the linear identity function
        Criterion: torch.nn.<Criterion> loss function corresponding
    
    Example:
        >>> preprocess, criterion = select_criterion("nll_loss")
        >>> train(n_epochs, loaders_scratch, model, optimizer, 
                  criterion, use_cuda, checkpoints[network], lr_scheduler, preprocess)
    '''
    # Linear identity function by default
    preprocess = lambda self : self
    if lossfn == 'cross_entropy':
        Criterion = nn.CrossEntropyLoss()
    elif lossfn == 'mse_loss':
        Criterion = nn.MSELoss()
    elif lossfn == 'nll_loss':
        Criterion = nn.NLLLoss()
        # NLLLoss expects input to be LogSoftMax values
        preprocess = lambda x : F.log_softmax(x, dim=1)
    else:
        print(fn + ' not recognized. Check input or add loss criterion to method. Selected default "cross_entropy".')
        Criterion = nn.CrossEntropyLoss()
    
    return Criterion, preprocess

### Implementations - `train`, `test`

In [14]:
def train(n_epochs, loaders, model, optimizer, criterion=nn.CrossEntropyLoss(),
                                               use_cuda=True,
                                               save_path='model_train_default.pt',
                                               lr_scheduler=None,
                                               preprocess = lambda self: self):
    """
    Model trainer.
    
    Args:
        n_epochs: numer of training iterations of the complete training set
        loaders: dataLoader with data for training
        model: model to train
        optimizer: backward propagation optimizer
        criterion: training criterion
        use_cuda: if cuda should be used for training
        save_path: save path for trained model
        lr_scheduler: learning rate scheduler for changing learning rates during training
        preprocess: preprocess network output for criterion (for more interactive modeling)
        
    Returns: trained model
    """ 
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU if possible
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            def closure():
                '''
                Compute the training error and gradient as input to optimizer. Required for 
                some optimizers (e.g. conjugate gradient and LBFGS).
                '''
                # Reset graditents for training batch
                optimizer.zero_grad()
                # Forward prop
                output = model(data)
                # preprocess criterion input
                output = preprocess(output)
                # Compute loss
                loss = criterion(output,target)
                # Backprop
                loss.backward()
                # Update global closure_loss - for training error
                global closure_loss
                closure_loss = loss
                return loss
            optimizer.step(closure)
            # Train loss
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (closure_loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(loaders['valid']):
                # move to GPU if possible
                if use_cuda:
                    data, target = data.cuda(), target.cuda()
                # forward pass
                output = model(data)
                # preprocess criterion input
                output = preprocess(output)
                # calculate loos
                loss = criterion(output, target)
                # Validation loss
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (closure_loss.data - valid_loss))

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## Save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            print('Current validation Loss: {:.6f} \tPrevious min validation Loss: {:.6f}'.format(
            valid_loss,
            valid_loss_min
            ))
            valid_loss_min = valid_loss
            print('Lowest validation score so far. Saving current model to: \t ' + re.search('checkpoints/(.*)', save_path).group(1))
            
        # Learning rate scheduler decreases learning rate when learning stagnates. Optional
        if lr_scheduler == None:
            pass
        else:
            lr_scheduler.step(valid_loss_min)
        
        # Test to see training progress. Wastefull here. Only set to true for testing.
        if TEST_IN_TRAIN == True:       # Global variable
            if (epoch % 5 == 0):
                print(' ################## \n ## Test at ' + str(epoch) + ' epochs:  ...')
                test(loaders, model, criterion, use_cuda)
                print(' ################## ')
            
    # return trained model
    return model

In [15]:
def test(loaders, model, criterion=nn.CrossEntropyLoss(),
                         use_cuda=True,
                         preprocess = lambda self: self):
    """
    Test model accuracy and print results.
    
    Args:
        loaders: DataLoader with data for testing
        model: model to test
        criterion: test criterion for accuracy
        use_cuda: if cuda should be used for training
        preprocess: preprocess network output function for criterion (used with criterion switching)
        
    Returns: None
    """
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loaders['test']):
            # move to GPU if possible
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # forward pass
            output = model(data)
            # preprocess criterion input
            output = preprocess(output)
            # calculate loss
            loss = criterion(output, target)
            # update average test loss 
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('Test Accuracy: %2d%% (%2d/%2d)\n' % (
        100. * correct / total, correct, total))

## CNN's

### Scratch CNN 1

In [ ]:
# Network for testing #VGG16 like model but simplified
class Net_1(nn.Module):
    def __init__(self, n_classes = 133): ## 133 classes in this problem
        super(Net_1, self).__init__()
        # Feature layers
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(4,4)
        )
        self.classifier = nn.Sequential(
            nn.Linear(25088,4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096,2048),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(2048,n_classes)
        )
        
    def forward(self, x):
        ## Define forward behavior
        x = self.features(x)
        # Flatten input for classifier
        x = x.view(-1, 25088)
        x = self.classifier(x)
        
        return x
    
#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net_1(n_classes)
# normally distributed weights
model_scratch.apply(weight_init_normal)

networks['SNet1'] = model_scratch

### Scratch CNN 2
Test of network and inspired by [Udacity Student Hub thread](https://study-hall.udacity.com/rooms/community:nd101:633452-project-300/community:thread-11891619222-594683?contextType=room) by user _Mahmoud H_ and code referenced in [Pastebin](https://pastebin.com/MBSxfqqy).

In [16]:
linear_inputs = 6272*8

# Simple network for testing. Shallower network that uses batch normalization
class Net_2(nn.Module):
    def __init__(self, n_classes = 133):  #n_classes = 133 for my problem
        super(Net_2, self).__init__()
        # Feature layers
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(16),
            
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(32),
            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(64),
        )
        self.classifier = nn.Sequential(
            nn.Linear(linear_inputs,2048),
            nn.ReLU(),
            nn.Dropout(),
            #nn.Linear(500,500),
            #nn.ReLU(),
            #nn.Dropout(),
            nn.Linear(2048,n_classes)
        )
        
    def forward(self, x):
        # Convolutions
        x = self.features(x)
        # Flatten input for classifier
        x = x.view(-1, linear_inputs)
        # Linear classifier
        x = self.classifier(x)
        
        return x
    
#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net_2(n_classes)
# normally distributed weights
#model_scratch.apply(weight_init_normal)

networks['SNet2'] = model_scratch

### Untrained VGG16

In [ ]:
# VGG16 model
model_scratch = models.vgg16(pretrained=False)

# Replicates VGG16 classifier, but changes outputs and adds nn.LogSoftMax for test loop compatibility
classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=n_classes, bias=True)
  )

# Update classifier
model_scratch.classifier = classifier
# normally distributed weights
model_scratch.apply(weight_init_normal)

networks['VGG16'] = model_scratch

### Untrained ResNet18

In [ ]:
# ResNet18
model_scratch = models.resnet18(pretrained=False)

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512, 512)),
    ('relu1', nn.ReLU()),
    ('drop1', nn.Dropout()),
    ('fc2', nn.Linear(512, 200)),
    ('relu2', nn.ReLU()),
    ('drop2', nn.Dropout()),
    ('fc3', nn.Linear(200,133))
]))

# Replace model last layer with this classifier
model_scratch.apply(weight_init_normal)

networks['ResNet18'] = model_scratch

## Train all untrained models sequentially

In [17]:
# Train loop
for network in networks:
    print('\n ########################################## \n #### \n ####  Working on ' + network + ' network. \n #### \n ########################################## \n')
    model = networks[network]
    # Move model to GPU if possible
    if use_cuda:
        model = model.cuda()
    
    # Preprocess output and criterion functions
    criterion, preprocess = select_criterion(loss_fn)
    # Optimizer
    optimizer = optim.Adam(model_scratch.parameters(), lr = lr)
    # Change change learning rate if plateaus
    lr_scheduler = None# optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True)
    
    # Checkpoint for best model (validation) for this network an number of epochs
    checkpoints[network] = "checkpoints/model_scratch_" + network + '-' + loss_fn +  '-' + str(n_epochs) + '_epochs.pt'
    
    model = train(n_epochs, loaders_scratch, model, optimizer, 
                  criterion, use_cuda, checkpoints[network], lr_scheduler, preprocess)
    
    # Clean up models
    del model, preprocess, criterion, optimizer, lr_scheduler # Hopefully prevents CUDA from running out of memory all the time


 ########################################## 
 #### 
 ####  Working on SNet2 network. 
 #### 
 ########################################## 

Epoch: 1 	Training Loss: 5.671856 	Validation Loss: 4.925028
Current validation Loss: 4.925028 	Previous min validation Loss: inf
Lowest validation score so far. Saving current model to: 	 model_scratch_SNet2-cross_entropy-200_epochs.pt
Epoch: 2 	Training Loss: 4.889029 	Validation Loss: 4.990131
Epoch: 3 	Training Loss: 4.852938 	Validation Loss: 4.726913
Current validation Loss: 4.726913 	Previous min validation Loss: 4.925028
Lowest validation score so far. Saving current model to: 	 model_scratch_SNet2-cross_entropy-200_epochs.pt
Epoch: 4 	Training Loss: 4.852279 	Validation Loss: 4.823976
Epoch: 5 	Training Loss: 4.847648 	Validation Loss: 4.892925
Epoch: 6 	Training Loss: 4.828283 	Validation Loss: 4.806569
Epoch: 7 	Training Loss: 4.794207 	Validation Loss: 5.101537
Epoch: 8 	Training Loss: 4.770022 	Validation Loss: 4.781450
Epoch: 9 	Train

## Test trained models sequentially

In [18]:
for network in networks:
    print('\n ########################################## \n ####  Test ' + network + ' network. \n ########################################## \n')
    model = networks[network]
    # Get current checkpoint name
    checkpoint = checkpoints[network]

    if use_cuda:
        model = model.cuda()
    
    # Get loss function used to train network
    loss_fn = re.search('-(.*)-', checkpoint).group(1)
    
    # Initiate preprocess and criterion functions
    criterion, preprocess = select_criterion(loss_fn)
    # load the model that got the best validation accuracy
    model.load_state_dict(torch.load(checkpoint))
    
    # test function
    test(loaders_scratch, model, criterion, use_cuda, preprocess)
    
    # Hopefully will prevent CUDA from running out of memory all the time
    del model, preprocess, criterion


 ########################################## 
 ####  Test SNet2 network. 
 ########################################## 

Test Loss: 3.218578

Test Accuracy: 24% (205/836)



## Train and testing transfer learning - ResNet18

In [19]:
# Transfer learning base model architecture 
model_transfer = models.resnet18(pretrained=True)

# Freeze layers
for param in model_transfer.parameters():
    param.requires_grad = False
# Create classifier for classifying dog breed data
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512, 512)),
    ('relu1', nn.ReLU()),
    ('dropout1', nn.Dropout()),
    ('fc2', nn.Linear(512,133)),
]))

# Replace model last layer with this classifier
model_transfer.fc = classifier#.apply(weight_init_normal)

In [20]:
if use_cuda:
    model_transfer = model_transfer.cuda()

n_epochs = 20

loaders_transfer = loaders_scratch
# Loss and preprocess function (if any)
criterion_transfer, preprocess = select_criterion(loss_fn)
# Train only fc layers, other parameters are frozen
optimizer_transfer = optim.Adam(model_transfer.fc.parameters(), lr=lr)
# Modify learning rate on some criterion
lr_scheduler_transfer = optim.lr_scheduler.ReduceLROnPlateau(optimizer_transfer, 'min', verbose=True)

checkpoint = "checkpoints/model_transfer_pre_trained_ResNet18" + '-' + loss_fn +  '-' + str(n_epochs) + '_epochs.pt'

# train the model
model_transfer = train(n_epochs, loaders_transfer, model_transfer, optimizer_transfer, 
                       criterion_transfer, use_cuda, checkpoint, lr_scheduler_transfer, preprocess)

# load the model that got the best validation accuracy
model_transfer.load_state_dict(torch.load(checkpoint))

# Test pretrained model
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda, preprocess)

Epoch: 1 	Training Loss: 3.000970 	Validation Loss: 1.552920
Current validation Loss: 1.552920 	Previous min validation Loss: inf
Lowest validation score so far. Saving current model to: 	 model_transfer_pre_trained_ResNet18-cross_entropy-20_epochs.pt
Epoch: 2 	Training Loss: 1.571214 	Validation Loss: 1.121937
Current validation Loss: 1.121937 	Previous min validation Loss: 1.552920
Lowest validation score so far. Saving current model to: 	 model_transfer_pre_trained_ResNet18-cross_entropy-20_epochs.pt
Epoch: 3 	Training Loss: 1.353349 	Validation Loss: 1.396341
Epoch: 4 	Training Loss: 1.230539 	Validation Loss: 1.531572
Epoch: 5 	Training Loss: 1.159399 	Validation Loss: 0.593398
Current validation Loss: 0.593398 	Previous min validation Loss: 1.121937
Lowest validation score so far. Saving current model to: 	 model_transfer_pre_trained_ResNet18-cross_entropy-20_epochs.pt
 ################## 
 ## Test at 5 epochs:  ...
Test Loss: 0.677683

Test Accuracy: 79% (666/836)

 ############

# Run next notebook

In [21]:
#%run Udacity_dogbreed_code.ipynb